In [4]:
%load_ext autoreload
%autoreload 2

%matplotlib

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Using matplotlib backend: agg


In [70]:
from fastai.imports import *
from fastai.structured import *
import csv

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

In [15]:
df_raw = pd.read_csv('Bulldozer_train.csv', low_memory=False,
                     parse_dates=["saledate"])


In [16]:
df_raw.SalePrice = np.log(df_raw.SalePrice)

In [17]:
add_datepart(df_raw, 'saledate')
df_raw.saleYear.head()

0    2006
1    2004
2    2004
3    2011
4    2009
Name: saleYear, dtype: int64

In [18]:
train_cats(df_raw)

In [19]:
df_raw.UsageBand.cat.categories

Index(['High', 'Low', 'Medium'], dtype='object')

In [20]:
df_raw.UsageBand.cat.set_categories(['High', 'Medium', 'Low'], ordered=True, inplace=True)

In [21]:
df_raw.UsageBand = df_raw.UsageBand.cat.codes

In [22]:
os.makedirs('tmp', exist_ok=True)
df_raw.to_feather('tmp/bulldozers-raw')

In [23]:
df_raw = pd.read_feather('tmp/bulldozers-raw')

In [24]:
df, y, nas = proc_df(df_raw, 'SalePrice')

In [25]:
m = RandomForestRegressor(n_jobs=-1)
m.fit(df, y)
m.score(df,y)

/home/mendu/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9831742286045194

In [26]:
def split_vals(a,n): return a[:n].copy(), a[n:].copy()

n_valid = 12000  # same as Kaggle's test set size
n_trn = len(df)-n_valid
raw_train, raw_valid = split_vals(df_raw, n_trn)
X_train, X_valid = split_vals(df, n_trn)
y_train, y_valid = split_vals(y, n_trn)

X_train.shape, y_train.shape, X_valid.shape

((400698, 66), (400698,), (12000, 66))

In [27]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [31]:
# running random forest on the training set and finding the R-square
m = RandomForestRegressor(n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

/home/mendu/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


[0.1114619774902506, 0.39644481180212027, 0.9736774822869674, 0.7118883858950505]


In [32]:
df_trn, y_trn, nas = proc_df(df_raw, 'SalePrice', subset=30000, na_dict=nas)
X_train, _ = split_vals(df_trn, 20000)
y_train, _ = split_vals(y_trn, 20000)

In [33]:
m = RandomForestRegressor(n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

/home/mendu/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


[0.11315044014326174, 0.3894134670897751, 0.972555007814879, 0.7220176503694649]


In [56]:
preds = np.stack([t.predict(X_valid) for t in m.estimators_])
preds[:,0], np.mean(preds[:,0]), y_valid[0]

(array([10.389  , 10.71442, 10.43412, 10.59663, 10.49127, 10.66896, 10.5321 , 10.43412,  9.79813, 10.5321 ]),
 10.45908285502722,
 10.768484990022733)

In [64]:
b=[None]*12000

In [65]:
for i in range(0,12000):
    b[i]= np.mean(preds[:,i])

In [73]:
with open('Bull.csv','w') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(b)